# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

In [1]:
import pandas as pd
import numpy as np

url = 'https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv'
df_marketing_customer_analysis = pd.read_csv(url)
df_marketing = df_marketing_customer_analysis.copy()

In [2]:
df_marketing = df_marketing.drop(columns=["Unnamed: 0"])

In [3]:
df_marketing.head(3)

,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,Income,...,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size,Vehicle Type
0,DK49336,Arizona,4809.216960,No,Basic,College,2/18/11,Employed,M,48029,...,0.0,9,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize,NaN
1,KX64629,California,2228.525238,No,Basic,College,1/18/11,Unemployed,F,0,...,0.0,1,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize,NaN
2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2/10/11,Employed,M,22139,...,0.0,2,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize,A


In [4]:
def cleaning_column_names(df):

    # Change column names into lower case
    df.columns = df.columns.str.lower() # cannot use direcly columns.lower() because df.columns is Index object and immutable (cannot be modified).
                                        # so, we need to create a new Index objects rather dan modifying the existing ones.

    # Replace white spaces with _
    df.columns = df.columns.str.replace(' ','_') # cannot use df = df.columns.str.replace to modify the df
                                                # instead, need to assign the result back to the df.columns

    # Rename st with state
    df = df.rename(columns={"st":"state"}) # cannot use df.columns.str.rename because string methods doesnt have rename method

    return df

In [5]:
df_marketing = cleaning_column_names(df_marketing)

In [6]:
def cleaning_invalid_values(df):

    # standardized gender values
    gender_mapping = {'Femal': "F", 'female': "F", "Male": "M"}
    df.gender = df.gender.replace(gender_mapping) #cannot use map here because then if F and M already exist, it became NaN

    # Replace state abbreviations
    state_mapping = {"AZ":"Arizona", "Cali":"California", "WA": "Washington"}
    df.state = df.state.replace(state_mapping) #cannot use map here because then if F and M already exist, it became NaN

    # Rename "Bachelors" with "Bachelor"
    df.education = df.education.str.replace("Bachelors","Bachelor")

    # Delete % character in Customer Lifetime Value
    df.customer_lifetime_value = df.customer_lifetime_value.str.replace("%","")

    # Replace specific vehicle class
    luxury_vehicles = ["Sports Car", "Luxury SUV", "Luxury Car"]    
    for vehicle in df.vehicle_class:
        if vehicle in luxury_vehicles:
            df.vehicle_class = df.vehicle_class.replace(vehicle, "Luxury")

    return df

In [8]:
new_df_marketing = df_marketing[(df_marketing["total_claim_amount"] > 1000) & (df_marketing["response"] == "Yes")].reset_index(drop=True)

In [9]:
new_df_marketing.head(3)

,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,income,...,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type
0,OK31456,California,11009.13049,Yes,Premium,Bachelor,1/24/11,Employed,F,51643,...,0.0,1,Corporate Auto,Corporate L3,Offer2,Agent,1358.4,Luxury Car,Medsize,NaN
1,YJ16163,Oregon,11009.13049,Yes,Premium,Bachelor,1/24/11,Employed,F,51643,...,0.0,1,Special Auto,Special L3,Offer2,Agent,1358.4,Luxury Car,Medsize,A
2,GW43195,Oregon,25807.06300,Yes,Extended,College,2/13/11,Employed,F,71210,...,1.0,2,Personal Auto,Personal L2,Offer1,Branch,1027.2,Luxury Car,Small,A


2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

In [10]:
df_yes_response = df_marketing[(df_marketing["response"] == "Yes")]
new_df = df_yes_response.groupby(["policy_type", "gender"])["total_claim_amount"].mean().round(2)

In [11]:
new_df

policy_type     gender
Corporate Auto  F         433.74
                M         408.58
Personal Auto   F         452.97
                M         457.01
Special Auto    F         453.28
                M         429.53
Name: total_claim_amount, dtype: float64

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

In [12]:
customers_with_policy = df_marketing.groupby("state")["customer"].count()
filtered_df = customers_with_policy[customers_with_policy > 500]


In [13]:
customers_with_policy

state
Arizona       1937
California    3552
Nevada         993
Oregon        2909
Washington     888
Name: customer, dtype: int64

In [14]:
filtered_df

state
Arizona       1937
California    3552
Nevada         993
Oregon        2909
Washington     888
Name: customer, dtype: int64

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [18]:
summary_customer_lifetime_value = df_marketing.groupby(["education","gender"])["customer_lifetime_value"].agg(['max', 'min','median']).round(2)

In [19]:
summary_customer_lifetime_value

max      min   median
education            gender                            
Bachelor             F       73225.96  1904.00  5640.51
                     M       67907.27  1898.01  5548.03
College              F       61850.19  1898.68  5623.61
                     M       61134.68  1918.12  6005.85
Doctor               F       44856.11  2395.57  5332.46
                     M       32677.34  2267.60  5577.67
High School or Below F       55277.45  2144.92  6039.55
                     M       83325.38  1940.98  6286.73
Master               F       51016.07  2417.78  5729.86
                     M       50568.26  2272.31  5579.10

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9